In [195]:
import codecs
import jieba as jb
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [257]:
#中文分局
def SentsTokenizer4Ch(text):
    #sentences = re.split('(。|！|\!|\.|？|\?)',text)
    sentences = re.split('(。|！|\!|？|\?)',text)
    new_sents = []
    for i in range(int(len(sentences)/2)):
        sent = sentences[2*i] + sentences[2*i+1]
        sent = sent.replace('\r\n','')
        sent = sent.strip()
        new_sents.append(sent)
    return new_sents

#删除所有符号,只保留字母、数字和中文
def remove_punctuation(line):
    #line = str(line)
    if line.strip()=='':
        return ''
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = rule.sub('',line)
    return line
 
#中文停用词
def stopwordslist(filepath):  
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]  
    return stopwords  
 
#加载停用词
stopwords = stopwordslist("./data/chineseStopWords.txt")


In [301]:
myfile = "./data/chatbot-data/5G-ch.txt"
text = codecs.open(myfile, "r", "utf-8").read()
sent_tokens = SentsTokenizer4Ch(text)
sent_tokens[:10]


['5G网络是第五代移动通信网络，其峰值理论传输速度可达每秒数十Gb，比4G网络的传输速度快数百倍。',
 '举例来说，一部1G超高画质电影可在3秒之内下载完成。',
 '随着5G技术的诞生，用智能终端分享3D电影、游戏以及超高画质（UHD）节目的时代正向我们走来。',
 '三星电子通过研究和试验表明，在28GHz的超高频段，以每秒1Gb以上的速度，成功实现了传送距离在2Km范围内的数据传输。',
 '此前，世界上没有一个企业或机构开发出在6GHz以上的超高频段实现每秒Gb级以上的数据传输技术，这是因为难以解决超高频波长段带来的数据损失大、传送距离短等难题。',
 '三星电子利用64个天线单元的自适应阵列传输技术，使电波的远距离输送成为可能，并能实时追踪使用者终端的位置，实现数据的上下载交换。',
 '超高频段数据传输技术的成功，不仅保证了更高的数据传输速度，也有效解决了移动通信波段资源几近枯竭的问题。',
 '2014年5月13日，三星电子宣布，其已率先开发出了首个基于5G核心技术的移动传输网络，并表示将在2020年之前进行5G网络的商业推广。',
 '2016年8月4日，诺基亚与电信传媒公司贝尔再次在加拿大完成了5G信号的测试。',
 '在测试中诺基亚使用了73GHz范围内的频谱，数据传输速度也达到了现有4G网络的6倍。']

In [259]:
#删除除字母,数字，汉字以外的所有符号
df = pd.DataFrame(sent_tokens, columns=['sent'])
df['clean_set']=  df['sent'].apply(remove_punctuation)

#分词，并过滤停用词
# df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x)) if w not in stopwords]))
df['cut_sent'] = df['clean_set'].apply(lambda x: " ".join([w for w in list(jb.cut(x))]))
df.head()

,sent,clean_set,cut_sent
0,5G网络是第五代移动通信网络，其峰值理论传输速度可达每秒数十Gb，比4G网络的传输速度快数百倍。,5G网络是第五代移动通信网络其峰值理论传输速度可达每秒数十Gb比4G网络的传输速度快数百倍,5G 网络 是 第五代 移动 通信 网络 其 峰值 理论 传输速度 可 达 每秒 数十 Gb...
1,举例来说，一部1G超高画质电影可在3秒之内下载完成。,举例来说一部1G超高画质电影可在3秒之内下载完成,举例来说 一部 1G 超 高画质 电影 可 在 3 秒 之内 下载 完成
2,随着5G技术的诞生，用智能终端分享3D电影、游戏以及超高画质（UHD）节目的时代正向我们走来。,随着5G技术的诞生用智能终端分享3D电影游戏以及超高画质UHD节目的时代正向我们走来,随着 5G 技术 的 诞生 用 智能 终端 分享 3D 电影 游戏 以及 超 高画质 UHD...
3,三星电子通过研究和试验表明，在28GHz的超高频段，以每秒1Gb以上的速度，成功实现了传送距...,三星电子通过研究和试验表明在28GHz的超高频段以每秒1Gb以上的速度成功实现了传送距离在2...,三星电子 通过 研究 和 试验 表明 在 28GHz 的 超高频 段 以 每秒 1Gb 以上...
4,此前，世界上没有一个企业或机构开发出在6GHz以上的超高频段实现每秒Gb级以上的数据传输技术...,此前世界上没有一个企业或机构开发出在6GHz以上的超高频段实现每秒Gb级以上的数据传输技术这...,此前 世界 上 没有 一个 企业 或 机构 开发 出 在 6GHz 以上 的 超高频 段 实...


In [322]:
GREETING_INPUTS = ("你好", "hi", "有人", "在吗", "嗨","在不在","有人吗",'在','有人')
GREETING_RESPONSES = ["你好",  "我在", "请问,有什么可以帮您的吗?", "你好，我在", "你好，很高兴见到你！"]

def greeting(sentence):
    rule = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    text = rule.sub('',sentence)
    if text in GREETING_INPUTS:
        return random.choice(GREETING_RESPONSES)
    
    wordlist =  [w for w in jieba.cut(sentence)]
    for word in wordlist:
        if word in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [323]:
greeting('有人在吗？')

'你好，我在'

In [291]:
def response(user_response):
    robo_response=''
    user_response  = remove_punctuation(user_response)
    user_response= " ".join([w for w in list(jb.cut(user_response)) if w not in stopwords])
    cut_sent = df.cut_sent.values.tolist()
    cut_sent.append(user_response)
    tfidf = TfidfVectorizer()

    tfidf_vec = tfidf.fit_transform(cut_sent)

    cos_sim = cosine_similarity(tfidf_vec[-1], tfidf_vec)
    idx=cos_sim.argsort()[0][-2]
    flat = cos_sim.flatten()
    flat.sort()
    req_tfidf = flat[-2]

    if(req_tfidf==0):
        robo_response=robo_response+"对不起,我不理解您的意思,我还是个小白...!"
        return(robo_response)
    else:
        robo_response = robo_response+df.sent.values[idx]
        return(robo_response) 



In [324]:
flag=True
print("机器人: 我的名字叫小白. 我可以回答您关于5G的问题. 如果您想退出, 请输入:bye !")

while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='多谢' or user_response=='谢谢' ):
            flag=False
            print("机器人: 不用谢！")
        else:
            if(greeting(user_response)!=None):
                print("机器人: "+greeting(user_response))
                print()
            else:
                print("机器人: ",end="")
                print(response(user_response))
                print()
    else:
        flag=False
        print("小白: 再见! 欢迎再次光临!")    

机器人: 我的名字叫小白. 我可以回答您关于5G的问题. 如果您想退出, 请输入:bye !
你好
机器人: 请问,有什么可以帮您的吗?

什么是5G网络啊？
机器人: 三星在5G网络上的取代技术突破，将给全球5G网络研发带来活力，并推动其商业化进程，同时有助于5G网络技术国际标准的制定。

5G网络的传输速度怎么样？
机器人: 5G网络是第五代移动通信网络，其峰值理论传输速度可达每秒数十Gb，比4G网络的传输速度快数百倍。

5G下载电影快吗？
机器人: 举例来说，一部1G超高画质电影可在3秒之内下载完成。

上海什么时候有5G啊？
机器人: 中国移动将在杭州、上海、广州、苏州、武汉这五个城市开展5G外场测试。

5G的峰值有多少啊？
机器人: 2014年7月，爱立信宣布，在5G无线技术一项无线测试中，传输速度峰值达到了5Gbps。

5G什么时候能商用？
机器人: 根据工信部等部门提出的5G推进工作部署以及三大运营商的5G商用计划，我国将于2017年展开5G网络第二阶段测试，2018年进行大规模试验组网，并在此基础上于2019年启动5G网络建设，最快2020年正式推出商用服务。

华为的5G怎么样？
机器人: 华为在17日举行的一场3GPP RAN1 87次会议的5G短码讨论方案中，凭借59家代表的支持，以极化码（Polar Code）战胜了高通主推的LDPC及法国的Turbo2.0方案，拿下5G时代的话语权。

5G用的是什么技术啊？
机器人: 三星在5G网络上的取代技术突破，将给全球5G网络研发带来活力，并推动其商业化进程，同时有助于5G网络技术国际标准的制定。

你是个笨蛋
机器人: 对不起,我不理解你的意思,我还是个小白...!

bye
小白: 再见! 欢迎再次光临!
